In [5]:
import requests

In [6]:
import pandas as pd

In [7]:
CHICAGO_RESOURCE_URI = 'data.cityofchicago.org'
socrata_api_token = input('Enter the API token here:  ')
socrata_authorization_header = {'Content-Type': 'application/json', 
                                'X-App-Token': socrata_api_token}

def where_a_lot_between(dataset, parameter, first_value, second_value):
    """Complete function for multi-query reading of long paginated responses.
    :param dataset: datasets' four by four code
    :param parameter: str, the name of parameter for where ... between query
    :param first_value: minimum value for between
    :param second_value: maximum value for between
    :return: list of records
    """
    def data_chunk(url, params):
        response = requests.get(url, headers=socrata_authorization_header, 
                                params=params)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 201:
            raise RuntimeError("Request Processing")
        elif response.status_code == 400:
            raise RuntimeError("Bad Request")
        elif response.status_code == 401:
            raise RuntimeError("Unauthorized")
        elif response.status_code == 403:
            raise RuntimeError("Forbidden")
        elif response.status_code == 404:
            raise RuntimeError("Not Found")
        elif response.status_code == 429:
            raise RuntimeError("Too many Requests")
        elif response.status_code == 500:
            raise RuntimeError("Server Error")
        else:
            return None  # silently return nothing

    api_url = f'https://{CHICAGO_RESOURCE_URI}/resource/{dataset}.json'
    api_call = f'{api_url}?$where={parameter} between "{first_value}" and "{second_value}"'

    limit = 1000    # limit of the frame within the time window
    offset = 0
    data = []       # the accumulation of chunks for final result
    data_to_read_left = True

    while data_to_read_left:
        # api_frame = api_call + f'&$limit={limit}&$offset={offset}'
        pag_params = {'$limit': limit, '$offset': offset}
        new_chunk = data_chunk(api_call, pag_params)
        data.extend(new_chunk)      # Append the newly acquired records to the list
        # a way to detect 'has_more' (with probability of error 1/2000)
        if len(new_chunk) == limit:
            offset += limit
            # print('offset: ', offset) # show the man that something's going on.
        else:
            data_to_read_left = False
    return data

Enter the API token here:  9vuup8mJbMiy5pSjyOCw6iuWl


# Define the slice of the dataset

In [8]:
dataset = 'ydr8-5enu'  # permits dataset
start_dt    = pd.Timestamp('2019-01-01', tz='US/Central') # pandas was of doing it is much better
start_str   = start_dt.strftime('%Y-%m-%dT%H:%M:%S')

end_dt      = pd.Timestamp('2019-12-31', tz='US/Central')
end_str     = end_dt.strftime('%Y-%m-%dT%H:%M:%S')
# the column 'where' will be applied to
column      = 'issue_date'

# Request the slice, pre-process it and create a dataframe

In [9]:
response = where_a_lot_between(dataset, column, start_str, end_str)
# exclude what can not fit into a database table anyway
exclude_columns = ['location']  # 'xcoordinate, ycoordinate, latitude, longitude,

result = pd.DataFrame.from_records(response, exclude=exclude_columns) # exclude works here, that's why
result.head(n=3)

,id,permit_,permit_type,review_type,application_start_date,issue_date,processing_time,street_number,street_direction,street_name,...,contact_13_type,contact_13_name,contact_13_city,contact_13_state,contact_13_zipcode,contact_14_type,contact_14_name,contact_14_city,contact_14_state,contact_14_zipcode
0,3001302,100799159,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2019-01-01T00:00:00.000,2019-01-01T00:00:00.000,0,3900,W,43RD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3001305,100799160,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2019-01-01T00:00:00.000,2019-01-01T00:00:00.000,0,300,E,RANDOLPH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3001306,100799161,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2019-01-01T00:00:00.000,2019-01-01T00:00:00.000,0,955,W,CERMAK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Format

In [10]:
from numpy import datetime64

column_types = {'id': int, 'permit_': int, 'reported_cost': float,
                # dates to numpy datetime format
                'application_start_date': datetime64, # will be a {Timestamp} format in the DataFrame
                'issue_date': datetime64,
                'processing_time': float,
                # coordinates of location
                'xcoordinate': float, 'ycoordinate': float,
                'latitude': float, 'longitude': float}

result = result.astype(column_types, errors='ignore')
result.head(n=3)

,id,permit_,permit_type,review_type,application_start_date,issue_date,processing_time,street_number,street_direction,street_name,...,contact_13_type,contact_13_name,contact_13_city,contact_13_state,contact_13_zipcode,contact_14_type,contact_14_name,contact_14_city,contact_14_state,contact_14_zipcode
0,3001302,100799159,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2019-01-01,2019-01-01,0.0,3900,W,43RD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3001305,100799160,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2019-01-01,2019-01-01,0.0,300,E,RANDOLPH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3001306,100799161,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2019-01-01,2019-01-01,0.0,955,W,CERMAK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
